# Data Establishment
## Public Art Archive

DS 2023 Final Project

Mackenzie Kleczynski - etm8nu

December 2025

**Note:** This notebook uses Claude AI for learning support in API and JSON processing. See "Note on Data Acquisition & AI Use" section for details.

---

## Goal for This Notebook

1. Acquire and read in the data source(s).
2. Describe how to get the data.
3. Describe who produced the data and how.
4. Describe the data's features with a COLS table.

---

## Part 1: Understanding the Data Source

### What is the Public Art Archive?

The Public Art Archive (PAA) is a free, continually growing online database powered by the nonprofit Creative West, serving as the largest centralized resource for discovering and learning about public artworks in the U.S. and globally (Creative West, n.d.).

- **Website:** https://publicartarchive.org/
- **Coverage:** 30,699 public artworks across the U.S. and abroad as of December 2025.
- **Data Types:** Artist names, materials, work types, themes, creation dates, locations, coordinates, etc.

### Why am I Using It?

I want to understand Virginia's public art landscape:
- What materials dominate?
- What types of art are most common?
- Which cities have the most public art?
- What's the timeline of creation?
- Where are artworks geographically distributed?
- How does Virginia compare globally?

### Limitations to Consider

- **API Limitations:** No direct/accurate state filtering so I had to use geographic boundaries instead.
- **Missing Records:** Not a complete census as it only includes artworks in Public Art Archive (user uploaded).
- **Data Currency:** Data from December 2025 so newer artworks may not be included.
- **Collection Bias:** Likely overrepresents institutional art and grassroots public art may be underrepresented.

**Citation:** Creative West. (n.d.). *Public Art Archive*. Retrieved December 2025, from https://www.publicartarchive.org

---

## Part 2: Data Acquisition Strategy

The Public Art Archive API has **two types** of queries:

**Type 1: Aggregations (Counts Only)**
- Returns: `{"material": "paint", "count": 4766}`
- Problem: Already counted up globally so you can't filter by state
- Result: 30,699 total records (unusable for specific Virginia analysis)

**Type 2: Individual Records (Full Data)**
- Returns: Complete artwork objects with address, materials, artist, coordinates, etc.
- Benefit: Each record has location data attached so I can filter by state before aggregating
- Result: I can get true Virginia-only data

### My Approach

**I used both types strategically:**
- **Type 1 (Global Aggregations):** For comparing Virginia to global trends later
- **Type 2 (Virginia Individual Records):** For detailed Virginia-specific analysis

**Steps:**
1. Query API using geographic bounding box (Virginia's approximate rectangle)
2. Get individual records (with location attached)
3. Filter in Python by keeping only records where state == "Virginia"
4. Merge with geographic coordinates from combined dataset
5. Result: Virginia-only data with all fields including latitude/longitude

---

## Part 3: Acquisition Process

This section explains how I acquired two different types of data from the Public Art Archive API: global aggregations and Virginia individual records.

### Finding the API Request Using Developer Tools

To get data from the Public Art Archive, I needed to know how to communicate with their API. The website itself makes requests to the API, so I used this as a template.

I opened the website in Chrome and used Developer Tools (F12) to see what requests were happening behind the scenes. I watched the Network tab to see the exact format and parameters the API accepts. This gave me the correct way to structure my own requests. Instead of guessing how to talk to the API, I used what the website was already doing as a template.

### Data Type 1: Global Aggregations

**How I got it:**

1. I sent a query to the API with no geographic filter
2. Requested all aggregations (counts of each category)
3. The API returned totals across all ~30,699 public artworks in their database

### Data Type 2: Virginia Individual Records

**How I got it:**

#### Step 1: Geographic Bounding Box Filter

The API doesn't support a direct "state" filter. Instead, it filters by geographic location using latitude and longitude coordinates.

A bounding box is a rectangle drawn around Virginia:
- West: -83.644109
- East: -75.393376
- South: 36.575456
- North: 39.490960

The API checks each artwork's coordinates to see if it falls within this rectangle. If it does, it includes it.

**The limitation:** A rectangle doesn't perfectly match Virginia's shape. It also includes parts of Maryland, DC, West Virginia, and a few other areas. So I got 627 records instead of just Virginia records.

**Problems I Encountered:** I initially tried to use Virginia's exact shape (a polygon with 40+ coordinate points) instead of a rectangle. This would have been more precise. However, the API rejected this request because it only accepts rectangular bounding boxes, not complex polygons.

#### Step 2: Getting All Records

The API sends data in chunks of 150 records at a time, so I had to make 5 requests:
- Request 1: Records 1-150
- Request 2: Records 151-300
- Request 3: Records 301-450
- Request 4: Records 451-600
- Request 5: Records 601-627

I then combined them into one JSON file.

### Raw Data Files

- **`global_aggregations.json`** — All pre-counted aggregations across the entire Public Art Archive database
- **`virginia_records.json`** — 627 individual records from the Virginia geographic area (before state filtering)
- **`virginia_public_art_627_records.json`** — 627 individual records with geographic coordinates (geo field)

These files contain the data exactly as they came from the API, before any processing or filtering.

---

## Part 4: Processing Raw Data

### Loading Required Libraries

In [13]:
import json
import pandas as pd
import numpy as np

### Processing Global Aggregations

In [35]:
# Load global aggregations
with open('Data/Raw Data/global_aggregations.json', 'r') as file:
    global_agg = json.load(file)

# Extract aggregations
aggregations = global_agg['data']['searchArts']['aggregations']

# Initialize list to hold all DataFrames
dfs_global = []

# Process each category
for agg in aggregations:
    category_name = agg['name']
    hits = agg['hits']
    
    # Convert to DataFrame
    df = pd.DataFrame(hits)
    df = df[['value', 'count']].sort_values('count', ascending=False)
    df['category'] = category_name
    dfs_global.append(df)

# Combine all into one table
global_all = pd.concat(dfs_global, ignore_index=True)

# Reorder columns
global_all = global_all[['category', 'value', 'count']]

# Sort by category and count
global_all = global_all.sort_values(['category', 'count'], ascending=[True, False])

# Save as one CSV
global_all.to_csv('Data/Processed Data/global_all_data.csv', index=False)

global_all.head()

,category,value,count
3127,artists,Unknown Artist,303
3128,artists,Unknown,296
3129,artists,Richard Hunt,148
3130,artists,Dorothy Gillespie,106
3131,artists,Paul Caponigro,97


### Processing Virginia Individual Records

Loading and merging the two Virginia datasets to get complete individual records with coordinates:

In [39]:
# Load the Virginia data with all fields
with open('Data/Raw Data/virginia_records.json', 'r') as file:
    virginia_data = json.load(file)

df_new = pd.DataFrame(virginia_data['data']['searchArts']['items'])

# Load the Virginia data with geographic coordinates
raw_data_old = pd.read_json('Data/Raw Data/virginia_public_art_627_records.json')
data_dict = raw_data_old['data'].iloc[0]
df_old = pd.DataFrame(data_dict['items'])

# Filter both to Virginia only
df_new = df_new[df_new['address'].apply(lambda x: x.get('state') if x else None) == 'Virginia'].reset_index(drop=True)
df_old = df_old[df_old['address'].apply(lambda x: x.get('state') if x else None) == 'Virginia'].reset_index(drop=True)

# Merge on ID to combine all fields with coordinates
df_combined = df_new.merge(df_old[['id', 'geo']], on='id', how='left')

### Extracting and Cleaning Virginia Individual Records

In [40]:
# Extract and clean the combined data
virginia_records_df = []

for idx, record in df_combined.iterrows():
    virginia_records_df.append({
        'artwork_id': record.get('id'),
        'title': record.get('titles', [None])[0] if record.get('titles') else None,
        'artist': record.get('artists', [None])[0] if record.get('artists') else None,
        'address': record.get('address', {}).get('address') if isinstance(record.get('address'), dict) else None,
        'city': record.get('address', {}).get('city') if isinstance(record.get('address'), dict) else None,
        'state': record.get('address', {}).get('state') if isinstance(record.get('address'), dict) else None,
        'zip_code': record.get('address', {}).get('zip') if isinstance(record.get('address'), dict) else None,
        'latitude': record.get('geo')[1] if record.get('geo') and len(record.get('geo')) > 1 else None,
        'longitude': record.get('geo')[0] if record.get('geo') and len(record.get('geo')) > 0 else None,
        'materials': ', '.join(record.get('materials', [])) if record.get('materials') else None,
        'workType': ', '.join(record.get('workTypes', [])) if record.get('workTypes') else None,
        'themes': ', '.join(record.get('themes', [])) if record.get('themes') else None,
        'yearCreated': record.get('yearCreated'),
        'placement': ', '.join(record.get('placement', [])) if record.get('placement') else None,
        'collection': record.get('collections', [None])[0] if record.get('collections') else None
    })

virginia_df = pd.DataFrame(virginia_records_df)

# Save as CSV
virginia_df.to_csv('Data/Processed Data/virginia_records_filtered.csv', index=False)

virginia_df.head()

,artwork_id,title,artist,address,city,state,zip_code,latitude,longitude,materials,workType,themes,yearCreated,placement,collection
0,df9252f4-c491-4b84-97b1-83ff9b6bea9a,Sunflowers,Madeline Maas,162 Greenville Avenue,Staunton,Virginia,24401,38.147351,-79.071610,paint,mural paintings (visual works),None,2018,exterior walls,Augusta Christian Educators Studio Art Program
1,f62e0141-8697-4e03-952f-9c6b324b8cd3,Watering Can,Willy Ferguson,201 Greenville Avenue,Staunton,Virginia,24401,38.146625,-79.070925,carbon steel,outdoor sculpture,None,1999,median strips,"City of Staunton, Virginia"
2,df7cabbe-90ce-44d0-ab1b-2864f31f8ea4,American Flag Mural,Laura Whitmer,7014 Winterberry Ave.,Covington,Virginia,24426,37.792984,-79.905208,paint,mural paintings (visual works),None,2019,exterior walls,Kaylynn Farm
3,29ff85e0-4e4e-4acb-b457-993907b120fe,Columbarium Screen for Grace Evangelical Luthe...,Joan Huiner Ranzini,500 South Wayne Avenue,Waynesboro,Virginia,22980,38.063004,-78.894498,"steel (alloy), paint, concrete",outdoor sculpture,None,2010,religious buildings,Grace Evangelical Lutheran Church
4,1cf5aca9-2e2a-4f08-8ce2-035e522104de,RiverDance,Dimitar Lukanov,150 Keswick Court,Rockingham,Virginia,22801,38.389243,-78.828038,"stainless steel, paint",sculpture (visual work),None,2021,roadsides,Privately Owned


---

## Part 5: Data Dictionary (COLS Tables)

In [43]:
# Create base COLS table
virginia_cols_list = [(col.lower().split()[0], col) for col in virginia_df.columns]
COLS_VIRGINIA = pd.DataFrame(virginia_cols_list, columns=['col_id', 'col_name']).set_index('col_id')

# Add column types
COLS_VIRGINIA['col_type'] = 'measure'
COLS_VIRGINIA.loc[COLS_VIRGINIA.index.isin(['artwork', 'title', 'artist', 'address', 'city', 'state', 'collection']), 'col_type'] = 'identifier'
COLS_VIRGINIA.loc[COLS_VIRGINIA.index.isin(['materials', 'worktype', 'themes', 'placement']), 'col_type'] = 'categorical'

# Add unique count - use the actual column names
for col in COLS_VIRGINIA.index:
    actual_col_name = COLS_VIRGINIA.loc[col, 'col_name']
    COLS_VIRGINIA.loc[col, 'n_unique'] = virginia_df[actual_col_name].nunique()

COLS_VIRGINIA['n_unique'] = COLS_VIRGINIA['n_unique'].astype(int)
COLS_VIRGINIA

,col_name,col_type,n_unique
col_id,,,
artwork_id,artwork_id,measure,319
title,title,identifier,307
artist,artist,identifier,223
address,address,identifier,265
city,city,identifier,39
state,state,identifier,1
zip_code,zip_code,measure,60
latitude,latitude,measure,277
longitude,longitude,measure,278


In [44]:
# Create base COLS table
global_cols_list = [(col.lower().split()[0], col) for col in global_all.columns]
COLS_GLOBAL = pd.DataFrame(global_cols_list, columns=['col_id', 'col_name']).set_index('col_id')

# Add column types
COLS_GLOBAL['col_type'] = 'measure'
COLS_GLOBAL.loc[COLS_GLOBAL.index.isin(['category', 'value']), 'col_type'] = 'categorical'

# Add unique count
for col in COLS_GLOBAL.index:
    COLS_GLOBAL.loc[col, 'n_unique'] = global_all[col].nunique()

COLS_GLOBAL['n_unique'] = COLS_GLOBAL['n_unique'].astype(int)
COLS_GLOBAL

,col_name,col_type,n_unique
col_id,,,
category,category,categorical,9
value,value,categorical,16234
count,count,measure,368


---

## Note on Data Acquisition & AI Use

I included this section because I believe in being transparent about AI use, and I think it's really important to actually understand things before using AI to help you.

Before I started working with Claude AI on this project, I did my own research to understand APIs and how data collection works. I read through the Apify documentation on API scraping (https://docs.apify.com/academy/api-scraping), especially the GraphQL sections, and I also went through the GraphQL learning resources (https://graphql.org/learn/). I'm also taking DS 2022: Intro to Computing Systems with Daniel G. Graham right now, which helped me understand the basics of how data structures work.

Because I did this research first, I was able to have real conversations with Claude AI about what I was trying to do. When Claude helped me figure out how to work with the Public Art Archive API and merge the JSON files, I actually understood what was happening at each step. Through this process I was even able to ask more specific questions to understand fully about the data. 

For example, I was fascinated to learn about the bounding box process and was able to come up with and test ways I would try to navigate the API. For instance, I tested to see if I could use polygon points that I created in https://boundingbox.klokantech.com/. Even though this pathway attempt did not end up working with this API for this project, it was beneficial to learn about different approaches and limitations. When the polygon filtering failed, instead of just accepting that, I asked Claude why the API rejected it and what that meant about how the Public Art Archive was designed. Those conversations helped me understand that simpler APIs often don't support complex queries for performance reasons. I also had to troubleshoot issues like the missing latitude and longitude data, which led me to discover I needed to merge two different JSON files together. Each problem I ran into became a learning opportunity where I had to think critically about solutions instead of just copying code. Through these conversations I was able to more deeply understand where my data was coming from and be fully able to naturally understand and explain all of the concepts and processes.

I chose the harder path of collecting data directly from the API instead of using a pre-made Kaggle dataset because I'm genuinely interested in public art and Virginia, and this specific organization's work and collection. I wanted to really understand where the data came from.